In [2]:
# copyright ############################### #
# This file is part of the Xtrack Package.  #
# Copyright (c) CERN, 2021.                 #
# ######################################### #


import xtrack as xt
import xobjects as xo
import xpart as xp

import ducktrack as dtk
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from tqdm import tqdm






beta_rel=0.09405801936973575
gamma = 1.004453024014182

current=0.6
length = 2.5 # m cooler length
r_beam=25*1e-3



T_perp = 0.1 # <E> [eV] = kb*T
T_l =  0.01 # <E> [eV]
B = 0.075 # T for LEIR
B_ratio=1e-10


mass0=193729.0248722061*1e6 #eV/c2
c=299792458.0
p0c = mass0*beta_rel*gamma #eV/c

length = 78.54370266 #m
T = length/(c*beta_rel)
ms_per_turn = 1000*T



beta_x=5
beta_y=5
emittance=14*1e-6

disp_x=0


arc=xt.LinearTransferMatrix(Q_x=5.38518956965, Q_y=5.36921653748,
                            beta_x_0=beta_x, beta_x_1=beta_x,
                            beta_y_0=beta_y,  beta_y_1=beta_y,
                            alpha_x_0=0,   alpha_x_1=0,
                            alpha_y_0=0,   alpha_y_1=0,
                            disp_x_0=disp_x,disp_x_1=disp_x,
                            disp_y_0=0,    disp_y_1=0,
                            beta_s=1*1e40,
                            Q_s=0,
                            chroma_x=0.0, chroma_y=0
                                           
                 )



In [ ]:
num_particles=1000
#old
emittance=5*1e-6 #inital emittance
delta=5e-4

sigma_x = np.sqrt(beta_x*emittance)
sigma_px = np.sqrt(emittance*1/beta_x)
sigma_y = np.sqrt(beta_y*emittance)
sigma_py = np.sqrt(emittance*1/beta_y)
sigma_p = 5e-4


delta = np.random.normal(loc=0.0, scale=sigma_p,  size=num_particles)
x =     np.random.normal(loc=0.0, scale=sigma_x,  size=num_particles)+disp_x*delta
px =    np.random.normal(loc=0.0, scale=sigma_px, size=num_particles)
y =     np.random.normal(loc=0.0, scale=sigma_y,  size=num_particles)
py =    np.random.normal(loc=0.0, scale=sigma_py, size=num_particles)

# #1_sigma
# emittance1=5*1e-6 #inital emittance
# delta1=5e-4

#2_sigma
emittance_x_2sigma=35*1e-6 #inital emittance
x_2sigma=np.sqrt(beta_x*emittance_x_2sigma)
x_2sigma_beam=np.random.normal(loc=0.0, scale=x_2sigma,  size=num_particles)

delta_2sigma=1e-3
delta_2sigma_beam=np.random.normal(loc=0.0, scale=delta_2sigma,  size=num_particles)

dtk_particle = xp.Particles(
        
        mass0=mass0,
        p0c=p0c,
        x=disp_x*delta,
        px=0,
        y=0,
        py=0,
        delta=delta,
        zeta=0)


dtk_cooler = xt.ElectronCooler(current=current,length=length,r_beam=r_beam,
                                         T_perp=T_perp,T_l=T_l,
                                         B=B,B_ratio=B_ratio,
                                         Neutralisation_space_charge=1)

       

num_turns=int(1*1e5)
step=int(num_turns/100)

# initialize arrays
x, px, delta, action_x = [], [], [], []
file_index=0



# loop over turns
for i in tqdm(range(num_turns)):
        x.append(dtk_particle.x)
        px.append(dtk_particle.px)
        delta.append(dtk_particle.delta)
        
        # calculate action in horizontal plane
        action_x_temp = (dtk_particle.x**2/beta_x + beta_x*dtk_particle.px**2)
        action_x.append(action_x_temp)
    
        file_index+=1
        
        
        # track particle
        arc.track(dtk_particle)
        dtk_cooler.track(dtk_particle)



#time = np.arange(0, num_turns, step) * s_per_turn   

In [3]:
import json
# Load
with open('LEIR_2021_Pb_ions_for_tracking.json', 'r') as fid:
    loaded_dct = json.load(fid)
line_2 = xt.Line.from_dict(loaded_dct)



Done loading line from dict.           
